# CH 7 추천 시스템 평가
- 추천 시스템 평가는 크게 온라인 평가, 오프라인 평가, 사용자 스터디가 있음

## 7.1 3가지 평가 방법
#### 오프라인 평가
- 실제 서비스상에서의 열람, 구매 등 사용자 행동 이력에서 얻은 과거의 로그(서비스 로그)를 사용해 모델의 예측 정밀도 등을 평가
- 오프라인 평가는 성능이 좋았던 추천 모델을 사용해 배포해봐도 비즈니스 목표에 기여하지 않는 경우도 있음
- 비즈니스 목적의 대체 지표인 오프라인 평가 지표를 적절히 설정해야 함
- (이번장에서의)평가 지표 : 학습한 모델의 성능이나 예측값의 좋고 나쁨을 측정하는 지표

#### 온라인 평가
- 새로운 테스트 대상, 사용자 인터페이스를 일부 사용자에게 실제로 표시함으로써 평가를 수행
- 비즈니스 목표에 얼마나 기여했는지 오프라인과 비교해 정확한 평가를 수행하기 쉬움
- 출시까지 수행하는 구현 비용이 높고 아직 신규 추천 모델의 성능이 좋지 않은 경우 사용자 만족도를 떨어트릴 위험이 있음
#### 사용자 스터디
- 사용자에게 인터뷰나 설문을 하는 것으로 추천 모델이나 사용자 인터페이스의 정성적인 성질을 조사
- 서비스 로그만으로는 알 수 없는 개선점을 발견할 수 있음
- 개인 취향에 따른 답변의 분산이 크고 데이터의 양을 충분히 얻기 어려워 재현성이 떨어진다는 문제도 있음

## 7.2 오프라인 평가
- 모델 벨리데이션, 파라미터 튜닝, 평가 지표를 확인

### 7.2.1 모델 정밀도 평가
- 추천 시스템 목적은 과거 사용자 행동을 학습하고 미지의 사용자 행동을 높은 정밀도로 예측하는 것
- 모델의 일반화 성능 : 미지의 데이터에 대한 예측 능력

### 7.2.2 모델 벨리데이션
- 학습 데이터와 테스트 데이터로 나눔
- 추천 시스템 서비스 형태에 따라 아이템 소비에 관한 시계열 정보가 추천 정밀도에 크게 기여함
  - 서비스를 운영할 때 모델 학습에서 얻을 수 없는 미래 정보를 사용하는 학습은 피해야 함
  - 어떤 시점을 정해서 해당 시점 이전 데이터로 학습하고 이후 데이터로 테스트 데이터를 분할하는 등 적절한 데이터셋을 구축
- 학습 데이터, 검증 데이터, 테스트 데이터를 6: 2 :2 혹은 8 : 1 : 1로 분할하는 경우가 많음
- 미지의 데이터에 대해 일반화 성능을 검증하는 것을 밸리데이션이라고 함

### 7.2.3 모델 튜닝
- 예측 성능이 높아지도록 모델이 가진 파라미터를 조정하는 것
- 벨리데이션 데이터로 벨리데이션을 통해 수행
- 파라미터 튜닝은 아래의 3가지로 수행
  - 수동
  - 그리드 서치
    - 각 파라미터의 후보를 미리 작성하고 각 파라미터의 모든 조합에 대해 평가를 수행
  - 베이즈 최적화
    - 이전 검증 결과를 사용해 이후의 파라미터를 베이즈 확률 프레임에서 선택하는 방법
    - Optuna, Hyperopt 라이브러리 사용하여 수행

### 7.2.4 평가 지표
- 데이터셋 설계, 모델 학습 완료 이후에는 평가 지표를 사용해 추천 모델을 평가
- 평가 지표 : 학습 모델의 성능이나 예측값의 좋고 나쁨을 측정하는 지표
  - 예측 오차 지표
    - 학습 모델이 얼마나 테스트 데이터 평가값에 가깝게 예측할 수 있는지 측정
    - MAE, MSE, RMSE
  - 집합 평가 지표
    - 모델이 출력한 높은 점수의 아이템 집합 k개에 관한 추출 능력을 측정
    - Precision, Recall, F1-measure
  - 순위 평가 지표
    - 아이템 순서를 고려한 순위 평가에 사용
    - nDCG, MAP, MRR
  - 기타 평가 지표
    - 클릭 유무와 같은 예측 정밀도 외에 사용자 만족도를 직접 측정
    - 커버리지, 다양성, 신규성, 흥미로움(세렌디피티)
- 아래부터 설명된 지표의 소스는 src.metrics.py 파일에 있음

#### 7.2.4.1 예측 오차 지표
- 사용자가 아이템에 부여한 평가값과 시스템이 예측한 평가값의 오차를 평가하는 것
  - $r_i$: 실제 평갓값
  - $\widehat{r_i}$: 예측한 평갓값
  - $n$: 아이템 수
- 파이썬 구현
  ```python
    r = [0,1,2,3,4]
    r_hat = [0.1, 1.1, 2.1, 3.1, 4.1]
  ```

##### MAE
- 평균 절대값 오차 : 예측과 실측값 차이의 절대값 평균을 나타냄

##### MSE
- 평균 제곱 오차 : 예측과 실측값 차이의 제곱 평균

##### RMSE
- 평균 제곱근 오차 : MSE의 제곱근을 구해 예측과 실측값의 차원을 맞춤

#### 7.2.4.2 집합 평가 지표
- 예측 아이템 집합과 사용자가 실제로 클릭한 아이템 집합을 입력으로 하는 지표
- Precision, Recall, F1-measure 등이 있음
- 파이썬 설명
  - pred_items : 사용자가 클릭할 것이라고 예측한 아이템 집합(예측 아이템)
  - true_item : 사용자가 실제로 클릭한 아이템 집합(적합 아이템)
```python
# 예측 아이템 집합. 
pred_items = [1, 2, 3, 4, 5]
true_itmes = [2, 4, 6, 8]
```

##### Precision 
- 예측 아이템 집합 안에 존재하는 적합 아이템 비율
- 순위의 길이는 K로 표현
  - Precision@3 (Precision@K) : 예측 아이템 집합 상위 3개 중 1개의 아이템이 적합 = $1\over3$

##### Recall
- 예측 아이템 집합의 요소가 얼마나 적합 아이템 집합의 요소를 커버할 수 있는가의 비율
- Recall@3 (Recall@K) : 적합 아이템이 4개 있으며 예측 아이템 집합의 상위 3개 안에 아이템 B의 하나가 적합 아이템 = $1\over4$

##### F1-measure
- Precision과 Recall은 트레이드 오프 관계(반비례)
- F1-measure : Precision, Recall 양쪽을 고려해 평가하는 지표
  - Precision, Recall의 조화 평균으로 표현
  - F1@3 : Precision@3 = 1/3, Recall@3 = 1/4 일 때 0.286의 근사값으로 표현됨
  - (def f1_at_k)

#### 7.2.4.3 순위 평가 지표
- 추천 대상 아이템에 점수를 부여하여 점수가 높은 순으로 아이템을 정렬한 순위를 만듦
- 이 순위에 각 아이템의 배열을 고려하여 좋고 나쁨을 평가하는 지표

##### PR 곡선
- Top@K를 Top@1 ~ Top@N으로 바꾸어가며 Recall, Precision 조합을 확인
- Recall을 가로 축, Precision을 세로 축으로 플롯하여 각 점을 연결하는 것이 PR 곡선
- 플롯
![플롯1](./images/img_7-3.png)

- 그래프 형태로 표현
![플롯2](./images/img_7-2.png)

- Recall 0, Precision 1로 시작
- 모두 높은 영역은 플롯의 오른쪽 위에 위치하므로 이 곡선이 오른쪽 위로 위치할수록 정밀도가 높은 추천이라 할 수 있음
- PR곡선과 Recall, Precision의 각 축으로 둘러싸인 면적이 PR 곡선의 AUC(Area Under Curve)
- AUC는 0~1의 값을 가지며 1에 가까울수록 정밀도가 높음
- ROC 곡선의 AUC가 있음
- 일반적으로 ROC 곡선의 AUC는 음의 예와 양의 예보다 많을 경우 PR 곡선의 AUC를 보는 편이 적절하다고 생각할 수 있음

##### MRR@K
- 사용자 순위에 대해 최초의 적합 아이템이 순위에서 얼마나 상위에 위치하는지 평가하는 지표
- src.metrics.py 의 def rr_at_k

##### AP@K
- 평균 정밀도(Average Precision)는 순위의 K번째까지에 대해 각 적합 아이템까지의 정밀도를 평균한 값

##### MAP@K
- 중앙 평균 정밀도
- AP를 각 사용자에 대해 평균한 값
- MAP과 MRR은 식의 형태가 다르지만 다음 형식으로 일반화 할 수 있음

##### nDCG
- 지금까지는 클릭 유무와 같은 두 값에 대한 지표를 설명했음
- 클릭 이외에 구매 유무 등의 행동까지 합쳐서 가중치를 붙이고 싶을때 nDCG 사용
- DCG(Discounted Cumulative Gain)이라는 값을 정규화(nomalization)한 것

#### 7.2.4.4 기타 지표
- 지금까지의 소개는 사용자가 클릭한 아이템이 상위에 나타내기 위한 지표였음
- 위와 같은 지표는 필터 버블(비슷한 상품만 추천) 문제를 비롯해 서비스에서 바람직하지 않은 부작용을 일으킬 수 있음
- 정밀도 외에 측정 지표로 커버리지, 신규성, 다양성, 흥미로움 지표가 있음

##### 카탈로그 커버리지
- 추천 범위를 측정하는 지표
- 실제 추천한 상품 수 / 모든 상품의 수

##### 사용자 커버리지
- 어느 정도의 사용자에게 추천되었는지 측정하는 지표
- 실제로 추천이 수행된 사용자 집합 / 모든 사용자 집합
- 콜드 스타트 문제 검출에 사용 가능
- 시스템 이상 검출에도 도움이 됨

##### 신규성
- 순위에 대한 추천 아이템이 정말로 새로운지를 나타냄

##### 다양성
- 순위의 각 아이템 간 유사도 거리의 평균값에 따라 정의됨

##### 흥미로움
- 순위 중 의외성이 있는 아이템의 집합과 유용한 아이템의 집합의 교집합 / 순위 집합

#### 7.2.4.5 평가 지표 선정 방법
- 예측 정밀도, 다양성, 신규성 등의 관점이 있음
- 지금까지 나왔던 지표들을 온라인 지표와 관련된 오프라인 지표를 선택
- 또 다른 방법으로는 사용자 행동 이력을 가정해 지표를 선택
  - 유도형 : 특정 사이트를 방문하려는 의도
  - 정보 수집형 : 1개 이상의 웹 페이지에 기록되어 있다고 생각되는 정보를 얻으려는 의도
  - 거래형 : 웹을 매개로 한 액션을 실행하려는 의도
- 위 분류들은 추천 시스템 평가 시에도 유용
![평가 지표에 대한 행동 의도 분류](./images/img_7-5.png)

- ERR(Expected Reciprocal Rank)은 정보 검색 분야에서 유용한 지표 중 하나
- 주목한 아이템의 순위에 위치를 추가해 해당 아이템보다 위에 있는 아이템으로부터 정의된 정지 확률에 의존하는 지표

## 7.3 온라인 평가
- 모든 평가를 오프라인에서 정확하게 수행할 수는 없음
- 시스템의 변경점을 실제로 사용자에게 제시해 평가하는 방법을 온라인 평가라고 부름

### 7.3.1 A/B 테스트
![예시](./images/img_7-6.png)

- 대표적인 온라인 검증 방법
- 테스트 대상의 기능에 실험군과 대조군으로 나누어 평가를 수행

#### 7.3.1.1 가설
- 가설을 세우고 A/B 테스트를 진행하는 것이 좋음
- 각 항목의 의미는 다음과 같음
  - 콘텍스트 : 실험 배경은 무엇인가
  - 변경점 : 무엇을 어떻게 바꾸는가
  - 대상 : 대상 사용자는 누구인가
  - 지표 영향 : 변경점이 지표에 어떤 영향을 미치는가
  - 비즈니스 목표 : 달성하고 싶은 최종 비즈니스 목표는 무엇인가

##### 진행 방법
- 구현할 테스트를 실시함
- 사용자 그룹은 난수로 나눔

##### 주의점
- 테스트 전부터 그룹에 차이가 나지 않도록 (그룹 편향이 일어나지 않도록) 해야함
- 로그 혼합에도 주의해야 함
  - A 그룹에는 A 로그만, B 그룹에는 B 로그만으로 학습해야 함
- 집계 기간에도 주의해야 함
  - 특정 요일, 계절 조건에 따라 결과가 달라질 수 있음

#### 7.3.1.2 지표의 역할
- OEC, 가드레일 지표 소개와 주체적인 지표를 설계하는 지침에 관해 알아보자

##### OEC 지표
- A/B 테스트의 성공과 실패를 최종적으로 판단하는 지표
- 서비스/비즈니스 성공을 위해 시스템을 움직이는 것을 돕기 위하여 정의된 지표
- OEC 지표는 장기적인 서비스 KPI와 연관되어야 하며 동시에 단기간에는 팀이 행동할 때 충분히 공감해야 함
- OEC 지표는 사용자에게 초점을 맞춰 설계
  - Happiness
  - Engagement
  - Adoption
  - Retention
  - Task success

##### 가드레일 지표
- 저하되어서는 안되는 제약을 표시한 지표
  - 페이지 열람 수, 서비스 가동률, 응답 속도, 주간 액티브 사용자 수, 수익금 등

#### 7.3.1.3 지표 설계 방침
- STEDI 기준으로 어떤 지표를 선정하면 좋을지 판단

##### 감도 
- 업데이트에 대해 얼마나 빈번하게 변동하는가
  - 이동 확률 : 좋은 지표는 크게 변화할 확률이 적고 안정적으로 측정할 수 있음
  - 통계력 : 변동이 있을때 얼마나 정확하게 측정 가능한가

##### 신뢰성 
- 신뢰할 수 있는 지표를 정의하려고 노력해야 함 (데이터 신뢰, 

##### 효율성
- 지표는 효율이 좋은 의사결정으로 연결되어야 함

##### 디버깅 가능성과 액션 가능성
- 디버그 관련 지표 준비도 되어야 함
- 원인을 추적해 수정하는 거소가 연결되는 지표도 준비해야 함

##### 해석가능성과 방향성
- 쉽게 해석할 수 있어야 함(결과에 대한 모호한 해석은 지표 신뢰를 떨어트림)
- 방향성 : 해당 변수가 개선되었을 때 비즈니스 목표가 달성되는가를 나타냄

### 7.3.2 인터리빙
- 평가 대상의 각 순위를 하나의 순위로 섞어 사용자에게 제시
- 뒤섞인 순위에 대한 클릭으로 원래 순위끼리 평가
- 인터리빙 : 2개의 순위를 섞어 평가하는 방법
- 멀티리빙 : 3개의 순위를 섞어 평가하는 방법
![인터리빙과 AB테스트](./images/img_7-7.png)

- 대표적인 인터리빙의 특징은 아래와 같음
##### TDM
- 2개의 검토 결과를 선택할 때마다 선/후공을 무작위로 결정하며 양 순위에서 아직 사용되지 않은 검색 결과를 상위부터 순서대로 선택

##### PM 
- 각 순위 안의 검토 결과 순서를 유지하며 상대적으로 낮은 확률로 임의의 순서에서 검토 결과를 선택하도록 허용

##### OM 
- 먼저 출력 후보가 되는 순위를 다수 준비

##### PPM
- PPM은 속도가 빠르면서 이론적으로 보증하는 한편 구현이나 집계가 복잡하므로 TDM 부터 도입하는 것을 권장
- 인터리빙은 평가가 효율적이지만 UI등은 평가할 수 없음
- A/B 테스트에 비해 구현 비용이 상당히 높음

## 7.4 사용자 스터디를 통한 평가
- 지금까지들의 지표는 '실제 사용자가 추천 시스템을 어떻게 느끼고 있는지와 같이 서비스에 있어서 중요한 관점이 부족

### 7.4.1 조사 설계
- 사용자 스터디 설계 시 확인할 관점들에 대해 설명

##### 참가자 선정
- 실제 사용자에 가까운 사람을 선택
- 다양한 유형에 사용자를 대상으로 얻고 싶을 때 사용자를 분류하여 선정
  - 태스트 대상의 전문 지식 (초, 중, 상급)
  - 서비스 사용 빈도
  - 사용자 속성 정보

##### 참가자 수
- 조사 목적과 오차 허용 정도에 따라 결정됨
- 오류 발견의 목적이라면 소수
- 다양성 측명에서는 다수

##### 조사 시점
- 사용자 스터디 수행 시점은 다양하게 고려할 수 있음

##### 피험자 내 측정 / 피험자 간 측정
- 참가자별로 여러 개 얻은 데이터를 비교하는 피험자 내 측정이 있고
- 특정 참가자의 데이터를 다른 참가자의 데이터와 비교하는 피험자 간 특정이 있음

### 7.4.2 설문 조사 예
- 설문 조사를 통해 사용자 중심으로 평가를 수행하는 프레임워크 ResQue 소개
- ResQue 전체 이미지
![ResQue](./images/img_7-8.png)

- 시스템 품질 : 사용자가 시스템 품질을 어떻게 느꼈는가?
  - 적합성
    - 추진된 아이템에 흥미를 느낀다
  - 신선함
    - 추천된 아이템은 새로우며 흥미롭다
  - 다양성
    - 추천된 아이템에는 다양성이 있다
- 사용자의 신념 : 사용 용이성 등 시스템 품질의 결과로 얻어진 사용자의 신념
  - 추천 시스템의 인터페이스 레이아웃은 매력적이며 충분하다
  - 추천 시스템은 나에게 이상적인 아이템을 찾는 것을 효과적으로 도왔다
- 사용자의 태도 : 신뢰도나 만족도 같은 사용자의 주관적인 시스템에 대한 태도
  - 추천 시스템에 전체적으로 만족한다
  - 추천된 아이템은 납득할 수 있다
  - 추천된 아이템을 좋아하게 될 것이라 확신한다
- 행동 의도 : 사용자가 일으킨 행동
  - 지속성과 빈도
    - 추천 시스템을 다시 사용하고 싶다
  - 주변에 추천
    - 이 추천 시스템을 주변에 소개하고 싶다
